Importing libraries

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd

Reading files into dataframe

In [9]:
df = pd.read_csv("DEVICE_DIRECTORY.csv")
subdf = pd.read_csv("SUB_DEVICE_DIRECTORY.csv")
pd.set_option("display.max_columns", None)

In [ ]:
df['RAM_SIZE'].unique()

In [11]:
df.to_csv('DEVICE_DIRECTORY.csv', index=False)


In [ ]:
subdf

Converting 'DATE' columns into datetime datatype

In [ ]:
subdf['START_DATE'] = pd.to_datetime(subdf['START_DATE'])
subdf['END_DATE'] = pd.to_datetime(subdf['END_DATE'])
# subdf.to_csv('SUB_DEVICE_DIRECTORY.csv', index=False)
(subdf[subdf["ID"] == 1052312]).sort_values(by='START_DATE')


Filter brands from the average days used per brand dataset, sorted in descending order.


In [14]:
avg_days_used_per_brand = subdf.groupby('BRAND')['DAYS_USED'].mean().reset_index()
avg_days_used_per_brand.columns = ['BRAND', 'AVG_DAYS_USED']
pd.set_option("display.max_rows", None)
avg_days_used_per_brand.sort_values(by="AVG_DAYS_USED", ascending=False)
filtered_brands = avg_days_used_per_brand[avg_days_used_per_brand['BRAND'].isin(
    ['Apple', 'Samsung','Nokia','Xiaomi','Huawei','Motorola','Microsoft'])]

In [ ]:
subdf.columns
    

Addding a new column with the rounded average days used for each model and brand combination in the dataset.


In [17]:
subdf['AVG_DAYS_USED'] = (subdf.groupby(['MODEL', 'BRAND'])['DAYS_USED'].transform('mean')).round(1)
subdf.to_csv('SUB_DEVICE_DIRECTORY.csv', index=False)

Merge `subdf` with `df` on `DEVICE_ID` and `TAC`, including all rows from `subdf` and matching rows from `df`.


In [ ]:
merged_df = pd.merge(subdf, df, left_on='DEVICE_ID', right_on='TAC', how='left')
merged_df.sample(6)

In [ ]:
merged_df.head()

In [ ]:
merged_df.sample(12)

In [ ]:
merged_df.columns

Calculating the post-release purchase period by subtracting the `RELEASE_DATE` from the `START_DATE` and storing it in a new column.


In [26]:
merged_df['POST_RELEASE_PURCHASE'] = pd.to_datetime(merged_df['START_DATE']) - pd.to_datetime(merged_df['RELEASE_DATE'])

In [ ]:
merged_df.head()

Converting the `POST_RELEASE_PURCHASE` column to integers by extracting the number of days as a string and then converting it to an integer.


In [35]:
merged_df['POST_RELEASE_PURCHASE'] = merged_df['POST_RELEASE_PURCHASE'].astype(str).str.extract('(\d+)').astype(int)


In [36]:
merged_df = merged_df.rename(columns={'OS_TYPE_x': 'OS_TYPE'})


In [37]:
merged_df = merged_df[['ID', 'DEVICE_ID', 'BRAND', 'MODEL', 'NETTYPE', 'OS', 'OS_TYPE', 'OS_VENDOR',
        'RELEASE_DATE','START_DATE', 'END_DATE', 'POST_RELEASE_PURCHASE', 'DAYS_USED', 'AVG_DAYS_USED', 'SUPPORTS_LTE',
        'SUPPORTS_VOLTE', 'BAND_1800', 'BAND_1900', 'BAND_850', 'BAND_900',
        'CAMERA_FLASHLIGHT', 'SUPPORTS_MULTISIM', 'SUPPORTS_ESIM', 'SUPPORT_5G',
        'RAM_SIZE', 'SIM_COUNT', 'CPU_CORES', 'BATTERY_CAPACITY', 'BATTERY_TYPE', 
        'BODY_DEPTH', 'BODY_HEIGHT', 'BODY_WIDTH', 'BODY_TYPE',
         'MULTISIM_MODE', 'PIXEL_DENSITY']]

In [ ]:
merged_df.shape

Converting the `merged_df` DataFrame to a Dask DataFrame with 12 partitions and saving it as a single CSV file named `MERGED_DATA_PART.csv`.


In [ ]:
ddf = dd.from_pandas(merged_df, npartitions=12) 
ddf.to_csv('MERGED_DATA_PART.csv', index=False, single_file=True)

In [ ]:
ddf.head()

In [43]:
total_data = pd.read_csv('MERGED_DATA_PART.csv')

In [ ]:
total_data.head()